In [ ]:
# 必要なライブラリをインストール
!pip uninstall mecab-python3 fugashi
!pip install mecab-python3==1.0.5 fugashi[unidic-lite]
!pip install transformers
!pip install jaconv
!pip install ginza
!pip install spacy
!python -m spacy download ja_ginza
!pip install sentence-transformers
!python -m spacy download ja_core_news_sm
!pip install janome
!pip install mojimoji
!pip install openai fugashi unidic-lite

##日本語でやってみた（未完成）

###NLTKデータのダウンロード

In [ ]:
import nltk

# NLTKデータをダウンロード
nltk.download('punkt')
nltk.download('punkt_tab')

###日本語BERTモデルのセットアップ

In [ ]:
# 必要なモジュールをインポート
from transformers import pipeline

# 日本語BERTモデルのセットアップ
def setup_model():
    return pipeline("fill-mask", model="cl-tohoku/bert-base-japanese")

model = setup_model()
print("モデルのセットアップが完了しました！")


###Fugashiを用いた日本語のトークン化

In [ ]:
from fugashi import Tagger

# 日本語トークナイザーをセットアップ
tagger = Tagger()

def tokenize_japanese(sentence):
    return [word.surface for word in tagger(sentence)]

###ひらがなに変換する処理の追加

In [ ]:
import jaconv

def convert_to_hiragana(sentence):
    # 漢字やカタカナをひらがなに変換
    return jaconv.hira2kata(jaconv.kata2hira(sentence))

###リポグラム化処理の関数

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import re

# BERTモデルとトークナイザーの準備
tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
model = BertForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese")

# 禁止された文字を含む単語を置き換える関数
def lipogram_sentence_dev(sentence, model, forbidden_chars):
    """
    特定の文字を含む単語を特定の文字を含まない単語に置き換える
    :param sentence: 入力文
    :param model: BERTモデル
    :param forbidden_chars: 含んではいけない文字のリスト
    :return: リポグラム化された文
    """
    # 文をトークン化
    words = sentence.split()  # 簡易的な分割、必要に応じて形態素解析を使う

    replaced_sentence = []

    for word in words:
        # 禁止された文字を含む単語をチェック
        if any(char in word for char in forbidden_chars):
            # 単語をマスク
            masked_sentence = sentence.replace(word, "[MASK]")

            # トークン化
            inputs = tokenizer(masked_sentence, return_tensors="pt")
            with torch.no_grad():
                outputs = model(**inputs)

            # 予測されたトークンの確率を取得
            predictions = outputs.logits[0, inputs['input_ids'][0] == tokenizer.mask_token_id]
            predicted_token_id = torch.argmax(predictions).item()

            # 予測されたトークンを取得
            predicted_token = tokenizer.decode([predicted_token_id])

            # 禁止された文字を含まないトークンを選択
            if not any(char in predicted_token for char in forbidden_chars):
                replaced_sentence.append(predicted_token)
            else:
                replaced_sentence.append(word)  # 適切な候補がなければ元の単語を使う
        else:
            # 禁止された文字を含まない単語はそのまま使用
            replaced_sentence.append(word)

    return " ".join(replaced_sentence)

# 実行例
sentence = "わたしはきのうのごご、がっこうでともだちとたのしいじかんをすごしました。"
forbidden_chars = ['か', 'つ']  # 置き換え対象の語彙

result = lipogram_sentence_dev(sentence, model, forbidden_chars)
print("リポグラム化後:", result)

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import spacy

# SpaCy日本語モデルのロード
nlp = spacy.load("ja_core_news_sm")

# BERTモデルとトークナイザーの準備
tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
model = BertForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese")

# 禁止された文字を含む単語を置き換える関数
def lipogram_sentence_dev(sentence, model, tokenizer, forbidden_chars):
    """
    特定の文字を含む単語を特定の文字を含まない単語に置き換える
    :param sentence: 入力文
    :param model: BERTモデル
    :param tokenizer: BERTトークナイザー
    :param forbidden_chars: 含んではいけない文字のリスト
    :return: リポグラム化された文
    """
    # 形態素解析で単語に分割
    doc = nlp(sentence)
    words = [token.text for token in doc]

    replaced_sentence = []

    for word in words:
        # 禁止された文字を含む単語をチェック
        if any(char in word for char in forbidden_chars):
            # 単語をマスク
            masked_sentence = sentence.replace(word, "[MASK]", 1)

            # トークン化
            inputs = tokenizer(masked_sentence, return_tensors="pt")
            with torch.no_grad():
                outputs = model(**inputs)

            # 予測されたトークンの確率を取得
            mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
            predictions = outputs.logits[0, mask_token_index]

            # 上位候補を取得
            top_k = 10  # 上位10候補を取得
            top_k_ids = torch.topk(predictions, k=top_k, dim=-1).indices[0].tolist()
            candidates = [tokenizer.decode([idx]).strip() for idx in top_k_ids]

            # 禁止された文字を含まない単語を選択
            replacement = next((candidate for candidate in candidates if not any(char in candidate for char in forbidden_chars)), word)

            replaced_sentence.append(replacement)
        else:
            # 禁止された文字を含まない単語はそのまま使用
            replaced_sentence.append(word)

    return "".join(replaced_sentence)  # 日本語は単語間にスペースを挿入しない

# 実行例
sentence = "わたしはきのうのごご、がっこうでともだちとたのしいじかんをすごしました。"
# sentence = "私は昨日の午後、学校で友達と楽しい時間を過ごしました。"
forbidden_chars = ['か', 'つ', 'み', 'の']  # 置き換え対象の語彙

result = lipogram_sentence_dev(sentence, model, tokenizer, forbidden_chars)
print("リポグラム化後:", result)

In [ ]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from janome.tokenizer import Tokenizer

# BERTモデルとトークナイザーの準備
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
model = BertForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese")

# Janomeを使って単語ごとに分割する関数
def split_into_words(sentence):
    tokenizer = Tokenizer()
    return [token.surface for token in tokenizer.tokenize(sentence)]

# リポグラムのルールに従って後処理
def apply_lipogram_rules(word):
    # 小文字を大文字に、濁音・半濁音を清音に変換
    mapping = {
        'が': 'か', 'ぎ': 'き', 'ぐ': 'く', 'げ': 'け', 'ご': 'こ',
        'ざ': 'さ', 'じ': 'し', 'ず': 'す', 'ぜ': 'せ', 'ぞ': 'そ',
        'だ': 'た', 'ぢ': 'ち', 'づ': 'つ', 'で': 'て', 'ど': 'と',
        'ば': 'は', 'び': 'ひ', 'ぶ': 'ふ', 'べ': 'へ', 'ぼ': 'ほ',
        'ぱ': 'は', 'ぴ': 'ひ', 'ぷ': 'ふ', 'ぺ': 'へ', 'ぽ': 'ほ',
        'ぃ': 'い', 'ぇ': 'え', 'ぅ': 'う', 'ぉ': 'お', 'っ': 'つ', 'ー': 'う'
    }
    for old, new in mapping.items():
        word = word.replace(old, new)

    return word

def mask_forbidden_characters_words(sentence, forbidden_chars):
    """
    禁止文字を含む単語を[MASK]に置き換える
    """
    words = split_into_words(sentence)
    masked_sentence = []

    for word in words:
        # 禁止文字を含む単語を[MASK]に置き換え
        if any(char in word for char in forbidden_chars):
            masked_sentence.append("[MASK]")
        else:
            masked_sentence.append(word)

    return " ".join(masked_sentence)

def predict_replacement(sentence, model, tokenizer, forbidden_chars):
    """
    BERTモデルを用いて[MASK]を予測し、禁止文字を置き換える
    """
    masked_sentence = mask_forbidden_characters_words(sentence, forbidden_chars)
    inputs = tokenizer(masked_sentence, return_tensors="pt")

    # モデルで予測
    with torch.no_grad():
        outputs = model(**inputs)

    # [MASK]トークンの位置を取得
    mask_token_indices = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    predictions = outputs.logits[0, mask_token_indices]

    # 各[MASK]トークンを禁止文字を含まない予測トークンで置き換える
    replaced_sentence = masked_sentence
    for i, mask_index in enumerate(mask_token_indices):
        top_k = 10
        predicted_token_ids = torch.topk(predictions[i], top_k).indices.tolist()
        for token_id in predicted_token_ids:
            predicted_token = tokenizer.decode([token_id]).strip()
            if not any(char in predicted_token for char in forbidden_chars):
                replaced_sentence = replaced_sentence.replace("[MASK]", predicted_token, 1)
                break
        else:
            replaced_sentence = replaced_sentence.replace("[MASK]", "", 1)  # 適切な候補がない場合は空文字にする

    # リポグラムルールを適用して結果を修正
    final_sentence = " ".join([apply_lipogram_rules(word) for word in replaced_sentence.split()])

    return final_sentence

# 実行例
sentence = "つきよのよるにいちまいのてがみをかきました"
forbidden_chars = ['つ', 'よ', 'い', 'て']  # 禁止文字

# 実行
result = predict_replacement(sentence, model, tokenizer, forbidden_chars)
print("リポグラム化後:", result)


In [ ]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from janome.tokenizer import Tokenizer

# BERTモデルとトークナイザーの準備
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
model = BertForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese")

# Janomeを使って単語ごとに分割する関数
def split_into_words(sentence):
    tokenizer = Tokenizer()
    return [token.surface for token in tokenizer.tokenize(sentence)]

# 禁止文字を含む単語を[MASK]に置き換える関数
def mask_forbidden_characters_words(sentence, forbidden_chars):
    """
    禁止文字を含む単語を[MASK]に置き換える
    """
    words = split_into_words(sentence)
    masked_sentence = []
    masked_words = []  # 変換された単語を記録

    for word in words:
        # 禁止文字を含む単語を[MASK]に置き換え
        if any(char in word for char in forbidden_chars):
            masked_sentence.append("[MASK]")
            masked_words.append(word)  # 置き換えた単語を記録
        else:
            masked_sentence.append(word)

    return " ".join(masked_sentence), masked_words

# BERTモデルを用いて[MASK]を予測し、変換候補を表示する関数
def predict_replacement(sentence, model, tokenizer, forbidden_chars):
    """
    BERTモデルを用いて[MASK]を予測し、禁止文字を置き換える
    """
    masked_sentence, masked_words = mask_forbidden_characters_words(sentence, forbidden_chars)
    inputs = tokenizer(masked_sentence, return_tensors="pt")

    # モデルで予測
    with torch.no_grad():
        outputs = model(**inputs)

    # [MASK]トークンの位置を取得
    mask_token_indices = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    predictions = outputs.logits[0, mask_token_indices]

    # 各[MASK]トークンを予測し、変換候補を表示
    predicted_words = []
    for i, mask_index in enumerate(mask_token_indices):
        top_k = 10
        predicted_token_ids = torch.topk(predictions[i], top_k).indices.tolist()
        candidate_tokens = [tokenizer.decode([token_id]).strip() for token_id in predicted_token_ids]
        predicted_words.append(candidate_tokens)

    return masked_sentence, masked_words, predicted_words

# 実行例
sentence = "つきよのよるにいちまいのてがみをかきました"
forbidden_chars = ['つ', 'よ', 'い', 'て']  # 禁止文字

# 実行
masked_sentence, masked_words, predicted_words = predict_replacement(sentence, model, tokenizer, forbidden_chars)

print("元の文:", sentence)
print("変換前の文:", masked_sentence)
print("変換された単語:", masked_words)

for i, word in enumerate(masked_words):
    print(f"[MASK]に変換された単語: {word}")
    print(f"候補: {predicted_words[i]}")


In [ ]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from janome.tokenizer import Tokenizer
import mojimoji  # 漢字をひらがなに変換するためのライブラリ

# BERTモデルとトークナイザーの準備
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
model = BertForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese")

# Janomeを使って単語ごとに分割する関数
def split_into_words(sentence):
    tokenizer = Tokenizer()
    return [token.surface for token in tokenizer.tokenize(sentence)]

# 漢字をひらがなに変換する関数
def kanji_to_hiragana(word):
    # mojimojiで簡単なひらがな変換を行う（※実際にはもっと精度高い変換が必要）
    return mojimoji.han_to_zen(word)

# リポグラムのルールに従って後処理
def apply_lipogram_rules(word):
    # 小文字を大文字に、濁音・半濁音を清音に変換
    mapping = {
        'が': 'か', 'ぎ': 'き', 'ぐ': 'く', 'げ': 'け', 'ご': 'こ',
        'ざ': 'さ', 'じ': 'し', 'ず': 'す', 'ぜ': 'せ', 'ぞ': 'そ',
        'だ': 'た', 'ぢ': 'ち', 'づ': 'つ', 'で': 'て', 'ど': 'と',
        'ば': 'は', 'び': 'ひ', 'ぶ': 'ふ', 'べ': 'へ', 'ぼ': 'ほ',
        'ぱ': 'は', 'ぴ': 'ひ', 'ぷ': 'ふ', 'ぺ': 'へ', 'ぽ': 'ほ',
        'ぃ': 'い', 'ぇ': 'え', 'ぅ': 'う', 'ぉ': 'お', 'っ': 'つ', 'ー': 'う'
    }
    for old, new in mapping.items():
        word = word.replace(old, new)

    return word

def mask_forbidden_characters_words(sentence, forbidden_chars):
    """
    禁止文字を含む単語を[MASK]に置き換える
    """
    words = split_into_words(sentence)
    masked_sentence = []

    for word in words:
        # 禁止文字を含む単語を[MASK]に置き換え
        if any(char in word for char in forbidden_chars):
            masked_sentence.append("[MASK]")
        else:
            masked_sentence.append(word)

    return " ".join(masked_sentence)

def predict_replacement(sentence, model, tokenizer, forbidden_chars):
    """
    BERTモデルを用いて[MASK]を予測し、禁止文字を置き換える
    """
    masked_sentence = mask_forbidden_characters_words(sentence, forbidden_chars)
    inputs = tokenizer(masked_sentence, return_tensors="pt")

    # モデルで予測
    with torch.no_grad():
        outputs = model(**inputs)

    # [MASK]トークンの位置を取得
    mask_token_indices = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    predictions = outputs.logits[0, mask_token_indices]

    # 各[MASK]トークンを禁止文字を含まない予測トークンで置き換える
    replaced_sentence = masked_sentence
    for i, mask_index in enumerate(mask_token_indices):
        top_k = 10
        predicted_token_ids = torch.topk(predictions[i], top_k).indices.tolist()
        for token_id in predicted_token_ids:
            predicted_token = tokenizer.decode([token_id]).strip()

            # 漢字の場合、ひらがなに変換
            hiragana_predicted = kanji_to_hiragana(predicted_token)

            if not any(char in hiragana_predicted for char in forbidden_chars):
                replaced_sentence = replaced_sentence.replace("[MASK]", predicted_token, 1)
                break
        else:
            replaced_sentence = replaced_sentence.replace("[MASK]", "", 1)  # 適切な候補がない場合は空文字にする

    # リポグラムルールを適用して結果を修正
    final_sentence = " ".join([apply_lipogram_rules(word) for word in replaced_sentence.split()])

    return final_sentence

# 実行例
sentence = "つきよのよるにいちまいのてがみをかきました"
forbidden_chars = ['つ', 'よ', 'い', 'て']  # 禁止文字

# 実行
result = predict_replacement(sentence, model, tokenizer, forbidden_chars)
print("リポグラム化後:", result)


In [ ]:
# リポグラム化処理の関数
def lipogram_sentence(sentence, model, forbidden_chars):
    """
    特定の文字を含む単語を特定の文字を含まない単語に置き換える
    :param sentence: 入力文
    :param model: BERTモデル
    :param forbidden_chars: 含んではいけない文字のリスト
    :return: リポグラム化された文
    """
    words = tokenize_japanese(sentence)
    replaced_sentence = []

    for word in words:
        # 禁止された文字を含む単語をチェック
        if any(char in word for char in forbidden_chars):
            # マスクされた文章を作成
            masked_sentence = sentence.replace(word, "[MASK]", 1)

            # トークン化してモデルに入力
            inputs = tokenizer(masked_sentence, return_tensors="pt")

            # モデルに予測をさせる
            with torch.no_grad():
                outputs = model(**inputs)

            # 予測されたトークンを取得
            mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
            predictions = outputs.logits[0, mask_token_index]

            # 予測されたトークンを選択
            predicted_token_id = torch.argmax(predictions, dim=-1).item()
            predicted_token = tokenizer.decode([predicted_token_id])

            # 禁止された文字を含まないトークンを選択
            if not any(char in predicted_token for char in forbidden_chars):
                replaced_sentence.append(predicted_token)
            else:
                replaced_sentence.append(word)  # 適切な候補がなければ元の単語を使う
        else:
            replaced_sentence.append(word)  # 禁止された文字を含まない単語はそのまま使用

    return "".join(replaced_sentence)  # 文章を結合して返す

In [ ]:
# テスト用の文章
# sentence = "私は昨日の午後、学校で友達と楽しい時間を過ごしました。"
sentence = "わたしはきのうのごご、がっこうでともだちとたのしいじかんをすごしました。"
forbidden_chars = ["か", "つ", "み", "わ", "き"]  # 禁止文字

print("元の文:", sentence)
new_sentence = lipogram_sentence(sentence, model, forbidden_chars)
print("リポグラム化後:", new_sentence)


##英語でやってみた

In [ ]:
import spacy
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Load English spaCy model
nlp = spacy.load("en_core_web_sm")

# Function to tokenize English text using spaCy
def tokenize_english_with_spacy(sentence):
    doc = nlp(sentence)
    return [token.text for token in doc]

# Function to replace words with forbidden characters using a BERT model
def lipogram_sentence_english(sentence, model, tokenizer, forbidden_chars):
    """
    Replace words with forbidden characters with alternatives that do not contain forbidden characters.
    :param sentence: Input sentence
    :param model: BERT model
    :param tokenizer: BERT tokenizer
    :param forbidden_chars: List of forbidden characters
    :return: Lipogrammed sentence
    """
    words = tokenize_english_with_spacy(sentence)
    replaced_sentence = []

    for word in words:
        # Check if the word contains any forbidden character
        if any(char in word for char in forbidden_chars):
            # Mask the word in the sentence
            masked_sentence = sentence.replace(word, "[MASK]", 1)

            # Tokenize and input the masked sentence into the model
            inputs = tokenizer(masked_sentence, return_tensors="pt")

            # Make predictions using the model
            with torch.no_grad():
                outputs = model(**inputs)

            # Get the predicted token
            mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
            predictions = outputs.logits[0, mask_token_index]

            # Choose the predicted token that doesn't contain forbidden characters
            predicted_token = None
            for _ in range(predictions.size(-1)):
                predicted_token_id = torch.argmax(predictions, dim=-1).item()
                predicted_token = tokenizer.decode([predicted_token_id])

                # Check if the predicted token contains forbidden characters
                if not any(char in predicted_token for char in forbidden_chars):
                    break
                else:
                    # Remove the invalid token if it contains forbidden characters
                    predictions[:, predicted_token_id] = -float("inf")  # Mask the invalid token

            # If no valid token found, use the original word
            if not predicted_token or any(char in predicted_token for char in forbidden_chars):
                replaced_sentence.append(word)
            else:
                replaced_sentence.append(predicted_token)
        else:
            replaced_sentence.append(word)  # Keep the original word if it doesn't contain forbidden characters

    return " ".join(replaced_sentence)

# Initialize the BERT model and tokenizer for English
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

# Example sentence
sentence = "yesterday evening, i walked along the beach, listening to the sound of the waves and feeling the cool breeze on my face. the sunset painted the sky with shades of orange and pink, creating a breathtaking view. later, i met my family for dinner at a seafood restaurant, where we shared stories and enjoyed delicious dishes together. it was a peaceful and memorable evening."

# Forbidden characters (example: vowels including "o" and "i")
forbidden_chars = ["e", "a"]

# Apply the lipogram transformation
new_sentence = lipogram_sentence_english(sentence, model, tokenizer, forbidden_chars)

print("\nOriginal sentence:\n", sentence)
print("\nLipogrammed sentence:\n", new_sentence)

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

# BERT モデルとトークナイザーをロード
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

def lipogram_sentence_english(sentence, model, tokenizer, forbidden_chars, top_k=10):
    """
    指定された禁止文字を含む単語を BERT により適切な単語に置換し、自然なリポグラム文を生成する。

    :param sentence: 元の文章
    :param model: BERT 言語モデル
    :param tokenizer: BERT トークナイザー
    :param forbidden_chars: 禁止する文字のリスト
    :param top_k: 予測する上位候補の数
    :return: リポグラム化された文章
    """
    words = sentence.split()  # 単純なスペース区切りで単語を取得
    replaced_words = []

    for word in words:
        # 禁止文字が含まれるかチェック
        if any(char in word for char in forbidden_chars):
            # [MASK] で置換
            masked_sentence = sentence.replace(word, "[MASK]", 1)
            inputs = tokenizer(masked_sentence, return_tensors="pt")

            # 予測実行
            with torch.no_grad():
                outputs = model(**inputs)

            mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
            predictions = outputs.logits[0, mask_token_index]

            # top_k 個の候補を取得
            top_k_tokens = torch.topk(predictions, top_k).indices[0].tolist()

            # 禁止文字を含まない最適な候補を選択
            predicted_token = word  # 初期値は元の単語
            for token_id in top_k_tokens:
                candidate = tokenizer.decode([token_id])
                if not any(char in candidate for char in forbidden_chars):
                    predicted_token = candidate
                    break

            replaced_words.append(predicted_token)  # 置換後の単語をリストに追加
        else:
            replaced_words.append(word)  # 禁止文字を含まない場合は元の単語を保持

    return " ".join(replaced_words)

# 入力文
original_sentence = "yesterday evening, i walked along the beach, listening to the sound of the waves and feeling the cool breeze on my face."
# original_sentence = "yesterday evening, i walked along the beach, listening to the sound of the waves and feeling the cool breeze on my face. the sunset painted the sky with shades of orange and pink, creating a breathtaking view. later, i met my family for dinner at a seafood restaurant, where we shared stories and enjoyed delicious dishes together. it was a peaceful and memorable evening."

# 禁止文字
forbidden_chars = ["a"]

# 変換実行
lipogrammed_sentence = lipogram_sentence_english(original_sentence, model, tokenizer, forbidden_chars)

print("\nOriginal sentence:\n", original_sentence)
print("\nLipogrammed sentence:\n", lipogrammed_sentence)

###評価してみた

In [ ]:
from sentence_transformers import SentenceTransformer, util

# モデルのロード
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 比較する文章
# sentence1 = sentence
# sentence2 = new_sentence
sentence1 = original_sentence
sentence2 = lipogrammed_sentence

# 文章をベクトルに変換
embeddings1 = model.encode(sentence1, convert_to_tensor=True)
embeddings2 = model.encode(sentence2, convert_to_tensor=True)

# コサイン類似度の計算
cosine_score = util.pytorch_cos_sim(embeddings1, embeddings2)[0][0]

print(f"文章1と文章2の類似度: {cosine_score}")

#新バージョン

In [ ]:
!apt install -y mecab libmecab-dev mecab-ipadic-utf8
!pip uninstall -y fugashi unidic_lite
!pip install fugashi[unidic-lite]

!pip install fugashi
!pip install ipadic

In [ ]:
openai.api_key = "your-api-key"  # ご自身のAPIキーをここに設定

In [ ]:
import openai
from fugashi import Tagger
import random

MODEL_NAME = "gpt-4"

banned_chars = ["さ", "い"]  # 禁止文字

tagger = Tagger()

def katakana_to_hiragana(katakana):
    return ''.join(chr(ord(c) - 0x60) if 'ァ' <= c <= 'ン' else c for c in katakana)

def get_reading(token):
    try:
        # トークンの読み仮名（カナ）を取得
        reading = token.feature.kana
        if reading != '*' and reading:
            return katakana_to_hiragana(reading)
    except Exception as e:
        print(f"エラー: {e}")

    # 読み仮名が取得できなかった場合は、元の表層形を基に変換
    return katakana_to_hiragana(token.surface)

def get_pos(token):
    try:
        # 品詞がリストの0番目にある場合（Unidic）
        pos = token.feature.pos1
        if pos != '*' and pos:
            return pos
    except Exception as e:
        print(f"エラー: {e}")
    return "名詞"  # デフォルトとして名詞を返す

def contains_banned(text):
    return any(c in text for c in banned_chars)

def rewrite_token(original_word, context=None, previous_replacements=None, pos=None, max_attempts=5):
    for attempt in range(max_attempts):
        # OpenAI APIで新しい単語候補を生成
        prompt = f"""
        以下の単語「{original_word}」は、禁止文字「{'、'.join(banned_chars)}」を含むため、文脈に合った自然な表現に**単語単位**で言い換えてください。
        ・文の文脈：「{context}」
        ・対象の単語：「{original_word}」
        ・品詞：「{pos}」
        ・出力は置き換えた語句 **一単語** にしてください。
        ・出力する語句は必ず**{original_word}**と違う単語を出力して
        ・禁止文字「{'、'.join(banned_chars)}」を**表記にも読み（ひらがな）にも含まないこと**。
        """

        # OpenAI APIへのリクエスト（新しいインターフェース）
        response = openai.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,  # 少しランダム性を持たせる
            max_tokens=100
        )

        # 生成された候補
        candidate = response.choices[0].message.content.strip()

        # トークン化して読み仮名を取得する
        temp_token = list(tagger(candidate))[0]  # 変換後の単語を再トークン化
        candidate_reading = get_reading(temp_token)

        # 禁止文字が含まれていない場合に結果を返す
        if not contains_banned(candidate) and not contains_banned(candidate_reading):
            return candidate, candidate_reading

    # 変換候補が見つからない場合、元の単語を返す
    return original_word, get_reading(list(tagger(original_word))[0])

def rewrite_sentence_with_reading_check(sentence):
    tokens = list(tagger(sentence))
    new_tokens = []
    previous_replacements = set()  # 履歴を保持

    for token in tokens:
        surface = token.surface
        reading = get_reading(token)
        pos = get_pos(token)

        print(token)

        print(f"トークン：{surface}（読み：{reading}）")

        if contains_banned(surface) or contains_banned(reading):
            print(f"❌ 禁止文字を含む：{surface}（読み：{reading}）")
            # 置き換えを行う
            replacement, replacement_reading = rewrite_token(surface, sentence, previous_replacements=previous_replacements, pos=pos)

            # 置き換えた後に再度禁止文字に引っかかるかチェック
            while contains_banned(replacement) or contains_banned(replacement_reading):
                replacement, replacement_reading = rewrite_token(replacement, sentence, previous_replacements=previous_replacements, pos=pos)

            print(f"👉 「{surface}」→「{replacement}」（読み：{replacement_reading}）")
            new_tokens.append(replacement)
        else:
            new_tokens.append(surface)

    return ''.join(new_tokens)

# 🔵 テスト実行
input_text = "さるも木から落ちる。犬も歩けば棒に当たる"
print("🔵 元の文：", input_text)
output_text = rewrite_sentence_with_reading_check(input_text)
print("🟢 変換後：", output_text)


In [ ]:
import openai
from fugashi import Tagger
import random
import re

MODEL_NAME = "gpt-4"

# 禁止文字（表記・読み両方から排除）
banned_chars = ["さ", "い"]

# 形態素解析器（Unidic 推奨）
tagger = Tagger()

# カタカナ→ひらがな変換
def katakana_to_hiragana(katakana):
    return ''.join(chr(ord(c) - 0x60) if 'ァ' <= c <= 'ン' else c for c in katakana)

# 読み仮名取得
def get_reading(token):
    try:
        # Unidic の場合 feature.kana
        reading = getattr(token.feature, "kana", None) or getattr(token.feature, "reading", None)
        if reading and reading != "*":
            return katakana_to_hiragana(reading)
    except Exception as e:
        print(f"[読み取得エラー] {token.surface} : {e}")
    return katakana_to_hiragana(token.surface)

# 品詞取得（Unidic を前提とする）
def get_pos(token):
    try:
        pos = getattr(token.feature, "pos1", None)
        return pos if pos and pos != "*" else "名詞"
    except Exception as e:
        print(f"[品詞取得エラー] {token.surface} : {e}")
    return "名詞"

# 禁止文字を含むか
def contains_banned(text):
    return any(c in text for c in banned_chars)

# GPTによる言い換え生成（単語単位）
def rewrite_token(original_word, context=None, previous_replacements=None, pos=None, max_attempts=5):
    for attempt in range(max_attempts):
        prompt = f"""
以下の単語「{original_word}」は、禁止文字「{'、'.join(banned_chars)}」を含むため、文脈に合った自然な表現に**単語単位**で言い換えてください。
・文の文脈：「{context}」
・対象の単語：「{original_word}」
・品詞：「{pos}」
・出力は置き換えた語句 **一単語のみ** にしてください。
・絶対に説明文や補足は付けず、単語だけを出力してください。
・出力する語句は必ず**{original_word}**と異なる新しい単語であること。
・禁止文字「{'、'.join(banned_chars)}」を**表記にも読み（ひらがな）にも含まないこと**。
"""

        response = openai.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=100
        )

        candidate = response.choices[0].message.content.strip()

        # 不要な記号を削除し、一語だけ抽出
        candidate = re.sub(r'[「」『』"\'（）()［］\[\]]', '', candidate).split()[0]

        temp_token = list(tagger(candidate))[0]
        candidate_reading = get_reading(temp_token)

        if not contains_banned(candidate) and not contains_banned(candidate_reading):
            return candidate, candidate_reading

    # 失敗した場合は元の単語を返す
    fallback_token = list(tagger(original_word))[0]
    return original_word, get_reading(fallback_token)

# 文を処理し、禁止文字を含む単語を差し替える
def rewrite_sentence_with_reading_check(sentence):
    tokens = list(tagger(sentence))
    new_tokens = []
    previous_replacements = set()

    for token in tokens:
        surface = token.surface
        reading = get_reading(token)
        pos = get_pos(token)

        print(f"トークン：{surface}（読み：{reading}）")

        if contains_banned(surface) or contains_banned(reading):
            print(f"❌ 禁止文字を含む：{surface}（読み：{reading}）")
            replacement, replacement_reading = rewrite_token(surface, sentence, previous_replacements, pos)

            # 再チェック（最大5回）
            retry_count = 0
            while (contains_banned(replacement) or contains_banned(replacement_reading)) and retry_count < 5:
                replacement, replacement_reading = rewrite_token(replacement, sentence, previous_replacements, pos)
                retry_count += 1

            print(f"👉 「{surface}」→「{replacement}」（読み：{replacement_reading}）")
            new_tokens.append(replacement)
        else:
            new_tokens.append(surface)

    return ''.join(new_tokens)

# 🔵 テスト実行
input_text = "さるも木から落ちる。犬も歩けば棒に当たる"
print("🔵 元の文：", input_text)
output_text = rewrite_sentence_with_reading_check(input_text)
print("🟢 変換後：", output_text)


In [ ]:
from fugashi import Tagger

def katakana_to_hiragana(katakana):
    return ''.join([chr(ord(c) - 0x60) if 'ァ' <= c <= 'ン' else c for c in katakana])

def get_reading(token):
    # Unidic では読みは feature[7] にあります（ない場合は表層形を返す）
    features = token.feature
    if (features):
        reading = features[6] #読み仮名を代入
        print(features)
        if reading != "*" and reading:
            return katakana_to_hiragana(reading)
    return token.surface

# Unidic Lite を使う
tagger = Tagger("-d /usr/local/lib/python3.11/dist-packages/unidic_lite/dicdir")

sentence = "犬も歩けば棒に当たる"
for token in tagger(sentence):
    print(f"表層形：{token.surface}　読み：{get_reading(token)}")


In [ ]:
!apt install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 fugashi ipadic
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install.sh

In [ ]:
import fugashi
import ipadic

tagger = fugashi.GenericTagger(ipadic.MECAB_ARGS)
text = "私はAIが好きです"

for word in tagger(text):
    surface = word.surface
    features = word.feature
    print(f"{surface}: {features}")
